In [15]:
%matplotlib inline
#import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
import plotly.plotly as py
# plotly.tools.set_credentials_file(username='marced', api_key='7meNCtLRU4D2vUuHqCdP')
import plotly.graph_objs as go
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
import datetime

In [27]:
HW2=pd.read_csv('data/Trips/201605-hubway-tripdata.csv', delimiter=',', parse_dates=True)
wea2 = pd.read_csv('data/weather/weather16.csv', delimiter=',', index_col=[0], parse_dates=True)

In [31]:
wea2.columns

Index(['date', 'temp_high', 'temp_avg', 'temp_low', 'hum_high', 'hum_avg',
       'hum_low', 'wind_high', 'wind_avg', 'wind_low', 'precip'],
      dtype='object')

In [33]:
wea = pd.concat([wea2[['temp_avg','hum_avg','wind_avg']],
                pd.to_datetime(arg = wea2.date).dt.date,
                ],axis=1)
wea.head()

,temp_avg,hum_avg,wind_avg,date
0,37,58,13,2016-01-01
1,36,51,13,2016-02-01
2,38,56,12,2016-03-01
3,25,61,12,2016-04-01
4,17,43,10,2016-05-01


In [34]:
c = pd.DataFrame(0, index=np.arange(len(HW2)), columns=['hum','temp','wind'])
df = pd.concat([HW2[['gender', 'tripduration','birth year']], 
                pd.to_datetime(arg = HW2.starttime).dt.hour,
                pd.to_datetime(arg = HW2.starttime).dt.date,
                c, 
                HW2['start station name']],axis=1)

df.columns = ['gender', 'duration','age','hour','date', 'hum','temp','wind','station']
#HW.gender.replace(to_replace=0, value=np.nan,inplace=True)

In [35]:
df = df[np.isfinite(df['gender'])]
df = df[np.isfinite(df['age'])]
df = df[df['gender'] != 0]
df = df[df['duration'] < 3600*1]
df = df[df['duration'] > 4*60]
#df = df[np.isfinite(df['usertype'])]

In [36]:
for i in range(0,len(wea)):
    date = wea.date[i]
    a = np.where(df['date'] == date)[0]
    df.temp.iloc[a] = wea.temp_avg[i]
    df.hum.iloc[a] = wea.hum_avg[i]
    df.wind.iloc[a] = wea.wind_avg[i]
df.head(2)    

/Users/Marc-Ed/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,gender,duration,age,hour,date,hum,temp,wind,station
0,1,450,1981.0,0,2016-05-01,43,17,10,One Broadway / Kendall Sq at Main St / 3rd St
1,1,505,1989.0,0,2016-05-01,43,17,10,MIT Stata Center at Vassar St / Main St


In [61]:
binsTemp = pd.cut(df.temp, 6)
binsWind = pd.cut(df.wind, 6)
binsHum = pd.cut(df.hum, 6)

In [71]:
binsTemp.sort_values(inplace=True)
binsWind.sort_values(inplace=True)
binsHum.sort_values(inplace=True)

In [82]:
t = binsTemp.unique()
w = binsWind.unique()
h = binsHum.unique()

In [87]:
tempBin = []
windBin = []
humBin = []
for i in range(0,len(t)):
    tempBin.append(len(binsTemp[binsTemp == t[i]]))
    windBin.append(len(binsWind[binsWind == w[i]]))
    humBin.append(len(binsHum[binsHum == h[i]]))

In [103]:
t[1]

'(27.5, 38]'

In [106]:
#temperature
trace1 = go.Bar(
    x=[1,2,3,4,5,6],
    y=tempBin,
    opacity=1,
    marker=dict(
        color='orange'
    ),
    visible=True,
    name="Temperature classes",
    text = [t[0],t[1],t[2],t[3],t[4],t[5]]
)
trace2 = go.Bar(
    x=[1,2,3,4,5,6],
    y=windBin,
    opacity=1,
    marker=dict(
        color='red'
    ),
    visible=False,
    name="Wind classes",
    text = [w[0],w[1],w[2],w[3],w[4],w[5]]
)
trace3 = go.Bar(
    x=[1,2,3,4,5,6],
    y=humBin,
    opacity=1,
    marker=dict(
        color='green'
    ),
    visible=False,
    name="Humidity classes",
    text = [h[0],h[1],h[2],h[3],h[4],h[5]]
)

data = [trace1, trace2, trace3]

#BUTTONS
layout = Layout(
    xaxis=dict(
    title="classes (6 bins)"),
    title='Weather characteristics per class',
    updatemenus=list([
        dict(
            x=-0.05,
            y=1,
            yanchor='top',
            buttons=list([
                dict(
                    args=['visible', [True, False, False]],
                    label='Temperature class',
                    method='restyle',
                ),
                dict(
                    args=['visible', [False,True, False]],
                    label='Wind class',
                    method='restyle',
                ),
                dict(
                    args=['visible', [False, False,True]],
                    label='Humidity class',
                    method='restyle',
                ),
                dict(
                    args=['visible', [True, True,True]],
                    label='All',
                    method='restyle',
                ),
            ]),
        )
    ]),
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)